In [ ]:
import numpy as np
import pandas as pd
import random
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics



import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING)


import gc

#data = pd.read_table('C:/Users/pulzara/Documents/Datathon/dataset oficial/Modelar_UH2022.txt', delimiter = '|', encoding = "UTF-8")
data = pd.read_table('C:/Users/mhoughton/Downloads/Modelar_UH2022.txt', delimiter = '|', encoding = "UTF-8")

final = pd.DataFrame()
final2 = pd.DataFrame()
ids_list = []
dict_result = {}
dict_metricas = {}
id_errors = []

def resul(dict, ids_list):

    resultado = pd.DataFrame.from_dict(dict,orient='index')
    print(resultado)
    #resultado['ID'] = ids_list
    # resultado.set_index(ids_list['ID'])
    # resultado = resultado.drop('ID', 1)
    return  resultado
    
#data = pd.read_table('C:/Users/mhoughton/Downloads/Modelar_UH2022.txt', delimiter = '|', encoding = "UTF-8")
arr_random_ids = []

#def random_ids():
#    total_ids = 2  
#    for i in range(0, total_ids):
#        n = random.randint(437, 2746) 
#        arr_random_ids.append(n)
#
#    print(arr_random_ids)
#    return arr_random_ids

def random_ids():
    x = [random.randrange(437, 1500, 2) for p in range(0, 4)] 
    
    return x

#arr_random_ids =  random_ids()
#arr_random_ids = [2562] #596, 1860 #54 #130 #85
for ID in range(0, 2747):
    print(ID)
    def id(data, i, sort_value_column):

        """
        Esta funcion obtiene un numero de id de contador en especifico.
        Realiza un sort values
        """
        data = data[data.ID == i]
        data = data.sort_values(by= sort_value_column, ascending=True)
        data1 = data.reset_index(drop = True)  
        #print(data1)
        return data1 


    data1= data.copy()

    def hour(data1, column_name):

        """
        cambia la fecha y lo deja en una hora sin minutos y segundos. 
        data, "column_name"
        """
        data1[column_name] = data1[column_name].str[:-5]
        data1[column_name] = data1[column_name] + '00:00'
        return data1 

    hour(data1, "SAMPLETIME") 
    

    data1 = id(data1, ID, "SAMPLETIME")  
    
    def negativo(df):
        df["DELTAINTEGER"].mask(df["DELTAINTEGER"] < 0, 0, inplace = True)
        df["DELTATHOUSANDTH"].mask(df["DELTATHOUSANDTH"] < 0, 0, inplace = True)
        return "True"
    
    negativo(data1)

    def deltainteger(df): 

        """
        Esta funcion crea una nueva variable deltainteger que es la suma de DELTAINTEGER + DELTATHOUSAND
        Cambia a datetime la columna SAMPLETIME
        """
        
        df["SAMPLETIME"] = pd.to_datetime(df["SAMPLETIME"])
        df = df.replace(np.nan,0) 
        df['DELTATHOUSANDTH'] = df['DELTATHOUSANDTH'].astype(float).astype(int)
        df.sort_values(by="SAMPLETIME", inplace=True)
        df.reset_index(inplace=True, drop=True)
        df['DELTAINTEGER'] = df['DELTAINTEGER'].astype(str)
        df['DELTATHOUSANDTH'] = df['DELTATHOUSANDTH'].astype(str)
        df['deltainteger'] = df['DELTAINTEGER'] + '.' + df['DELTATHOUSANDTH'] 
        df['deltainteger'] = df['deltainteger'].astype(float)  
        df.drop(df.columns[[2,3,4,5]], axis = 1, inplace =True) 

        return df

    data1 = deltainteger(data1) 
    
    data_week = data1.copy()  # Guardamos una copia del dataset para realizar la prediccion semanal 

    def group(data, column_name, frec):

        """
        Esta funcion agrupa por serie de tiempo en dias para obtener la media de deltainteger
        """
        data = data.groupby([pd.Grouper(key=column_name, freq=frec), "ID"]).sum()
        data = data.reset_index()
        data["SAMPLETIME"] = pd.to_datetime(data["SAMPLETIME"]) 
        data["SAMPLETIME"] = data["SAMPLETIME"].sort_values(ascending=False)
        data = data.set_index(column_name)

        return data 

    data1 = group(data1, "SAMPLETIME", "1D")
    
    
    # Creamos funciones para detectar la cantidad de outliers y reemplazar los valores por fuera del cuartil 0.95 para ser reemplazados por la media
    
    def valores_iqr(df_in, col_name):
        median = df_in[col_name].median()
        q1 = df_in[col_name].quantile(0.25) # 25th percentile / 1st quartile
        q3 = df_in[col_name].quantile(0.75) # 7th percentile / 3rd quartile
        iqr = q3-q1 #Interquartile range
        minimum  = q1-1.5*iqr # The minimum value or the |- marker in the box plot
        maximum = q3+1.5*iqr # The maximum value or the -| marker in the box plot
        return median, q1, q3, iqr, minimum, maximum

    def texto_iqr(df_in, col_name):
        median, q1, q3, iqr, minimum, maximum = valores_iqr(df_in, col_name)
        text = f"median={median:.2f}, q1={q1:.2f}, q3={q3:.2f}, iqr={iqr:.2f}, minimum={minimum:.2f}, maximum={maximum:.2f}"
        return text

    def remove_outliers(df_in, col_name):
        _, _, _, _, minimum, maximum = valores_iqr(df_in, col_name)
        df_out = df_in.loc[(df_in[col_name] > minimum) & (df_in[col_name] < maximum)]
        return df_out

    def count_outliers(df_in, col_name):
        _, _, _, _, minimum, maximum = valores_iqr(df_in, col_name)
        df_outliers = df_in.loc[(df_in[col_name] <= minimum) | (df_in[col_name] >= maximum)]
        return df_outliers.shape[0]

    def replace_outlier(df_in, col_name):
        q1 = df_in[col_name].quantile(0.25)
        q3 = df_in[col_name].quantile(0.95)
        iqr = q3-q1 #Interquartile range
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        df_out = df_in.copy()
        outliers = ~df_out[col_name].between(fence_low, fence_high, inclusive=False)
        df_out.loc[outliers, col_name] = df_out.loc[~outliers, col_name].mean()

        return df_out

    def outliers(data):
        """
        Esta funcion muestra el boxplot de deltainteger y la cantidad de los valores outliers, y reemplaza los outliers por la media de la columna deltainteger

        """
        #sns.boxplot(x ='deltainteger', data = data)   
        #plt.xticks(rotation=45)
        #plt.xlim(000,700000)
        #plt.show()
        for k, v in data.items():
            q1 = v.quantile(0.25)
            q3 = v.quantile(0.95) 
            irq = q3 - q1
            v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
            perc = np.shape(v_col)[0] * 100.0 / np.shape(data)[0] 
            print("Columna %s outliers = %.2f%%" % (k, perc))
        if perc <= 5:
            data1 = replace_outlier(data, "deltainteger")

        else:
            data1 = data
        #sns.boxplot(x ='deltainteger', data = data1)
        return data1
    
    data1 = outliers(data1) 
    #print("NADA MÁS SALIR DE LOS OUTLIERS")
    # Separamos los días de la semana según si son finde o no
    def is_weekday(ds):
        date = pd.to_datetime(ds)
        return (date.weekday() < 5)

    def prophe(df, data_week):

        """
        Esta funcion predice los 10 valores pedidos por datathon, 7 dias + 2 semanas
        """
        proph = True
        i = df['ID'].unique()

        df.reset_index(inplace=True)
        df.drop(df.columns[[1]], axis = 1, inplace=True) 
        cols = df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        df = df[cols]
        df = df.rename(columns={"deltainteger":"y", "SAMPLETIME": "ds"}) 

        ## Cambiamos infinitos por nan y los eliminamos
        #df = df.replace([np.inf, -np.inf], np.nan)
        df = df.dropna()

        # Marcamos findes
        df['weekday'] = df['ds'].apply(is_weekday)
        df['weekend'] = ~df['ds'].apply(is_weekday)

        ## Eliminamos outliers
        # stddev = df['y'].std()
        # mean = df['y'].mean()
        # data_no_outliers = df[(df.y < (mean + 4 * stddev)) & (df.y > (mean - 4 * stddev))]
        data_no_outliers = df
        df1 = df
        #print(df) 
           
        
        ## PRIMERO VAMOS A COMPROBAR COMO DE BIEN ESTAMOS PREDICIENDO.
        ## VAMOS A PREDECIR LAS ÚLTIMAS DOS SEMANAS DEL DATASET ORIGINAL 

        # Checkeamos si existenfilas sin datos suficientes:
        if df.shape[0] >= 2:
            proph = True
        else:
            print("AQUI NO HACE EL PROPHET PORQUE NO HAY DATOS")
            proph = False

        if proph:
            holidays = pd.DataFrame({
                'holiday': 'cvalencia',
                'ds': pd.to_datetime(['2019-01-01',
                                      '2019-03-19',
                                      '2019-04-19',
                                      '2019-04-22',
                                      '2019-05-01',
                                      '2019-06-24',
                                      '2019-08-15',
                                      '2019-10-09',
                                      '2019-10-12',
                                      '2019-11-01',
                                      '2019-12-06',
                                      '2019-12-25']),
                'lower_window': 0,
                'upper_window': 1,
            })

            ## SACAMOS LA PREDICCIÓN DEL DATATHON PRIMERO
            print("MODELO DATATHON.")
            
            # Test and train modelo 1
            
            
            # Seleccionamos las fechas que necesitamos predecir
            
            timestamp = pd.Timestamp('2020-02-07')
            delta = timestamp - max(df1.ds) 
            delta2 = delta.days 
            
            # Preparamos el prophet
            m = Prophet(interval_width=0.95, daily_seasonality=True, holidays=holidays) 
            # Añadimos seasonalitys en función de si es finde o no
            m.add_seasonality(name='weekday', period=1, fourier_order=3, condition_name='weekday')
            m.add_seasonality(name='weekend', period=1, fourier_order=3, condition_name='weekend')
            # Añadimos todas las vacaciones de españa como holidays
            m.add_country_holidays(country_name='ES')
            m.fit(df1) 
            
            ### PREDICCIÓN PARA LOS SIGUIENTES 7 DÍAS
            future = m.make_future_dataframe(periods=delta2, freq="D")
            future = future[-7:]
            future.reset_index(inplace = True, drop = True) 
            future['weekday'] = future['ds'].apply(is_weekday)
            future['weekend'] = ~future['ds'].apply(is_weekday)
            
            forecast = m.predict(future)
            #print("PRIMER FORECAST: ", forecast[['ds', 'yhat']].tail())
            
            #m.plot(forecast)  
            #plt.show()
            
            forecast1 = forecast.copy()    #[(forecast["ds"] >= '2020-02-01') &  (forecast["ds"] <= '2020-02-07')]
            forecast1["yhat"].mask(forecast1["yhat"] < 0, 0, inplace = True) 
            forecast1.reset_index(drop=True, inplace=True)  
            
            frames = [forecast1[["ds", "yhat"]]] 
            result = pd.concat(frames)
            
            
            # Metrica de evaluacion primer modelo
            
            #number = int(df1.shape[0]) - 8
            p = int(int(df1.shape[0])/3)  

            if int(df1.shape[0]) <100:
                # samples = []
                # print("ESTE ES EL DF 1:#################################################################################################")
                # print(df1.shape)
                # print(df1)
                # for index, row in df1.iterrows():
                #     print("Itero rows")
                #     sampletime = row['ds']
                #     delta = row['y']
                #     print(row['ds'], row['ds'])
                #     samples.append(sampletime.date() + timedelta(days = 1))
                #     samples.append(sampletime.date() - timedelta(days = 1))
                #     df_add = pd.DataFrame({'ds': samples}, columns=['ds'])
                #     df_add['y'] = 0
                #     df1.append(df_add)
                # print(df_add)
                # print(df1)
                # print(df1.shape)
                number = 100 
            else:
                number = int(df1.shape[0]) - 8
                
            try:
                # Realizamos cross validation
                cv = cross_validation(m, initial = f'{number} days', period=f'{p} days', horizon = '7 days')    
                
                df_pm = performance_metrics(cv)
                
                # Obtenemos el valor de la metrica de la media de los 7 dias del RMSE*0.5
                metric = (df_pm[:7]["rmse"].sum())/int(df_pm.shape[0])
                metric = metric*0.5

                
                d = metric
                #print("El valor RMSE para el modelo semanal es: ", d) 

                
                
                ## SACAMOS LA PREDICCIÓN DE LAS siguientes DOS SEMANAS D
                
                #print("Modelo dos semanas despues ")
                # creamos un nuevo dataset
                
                
                # Organizamos el dataset por semanas
                data_week = group(data_week, "SAMPLETIME", "1W")
                data_week.reset_index(inplace=True, drop = False)
                data_week.drop(data_week.columns[[1]], axis = 1, inplace=True) 
                cols = data_week.columns.tolist()
                cols = cols[-1:] + cols[:-1]
                data_week = data_week[cols] 
                data_week = data_week.rename(columns={"deltainteger":"y", "SAMPLETIME": "ds"})  
                
                
                timestamp2 = pd.Timestamp('2020-02-14')
                delta3 = timestamp2 - max(data_week.ds)   
                delta3 = delta3.days 
                delta3= round(float((delta3%365)/7)) 
                
                
                #data_week['weekday'] = data_week['ds'].apply(is_weekday)
                #data_week['weekend'] = ~data_week['ds'].apply(is_weekday)  


                
                m2 = Prophet(interval_width=0.95, daily_seasonality=True, holidays=holidays)
                # Añadimos seasonalitys en función de si es finde o no
                #m2.add_seasonality(name='weekday', period=1, fourier_order=3, condition_name='weekday')
                #m2.add_seasonality(name='weekend', period=1, fourier_order=3, condition_name='weekend')
                # Añadimos todas las vacaciones de españa como holidays
                m2.add_country_holidays(country_name='ES')
                m2.fit(data_week)  
                ### PREDICCIÓN PARA LOS SIGUIENTES 17 DÍAS
                future2 = m2.make_future_dataframe(periods=delta3, freq="W") 
                ## agregamos los valores que necesitamos para la prediccion y obtener la metrica
                
                future2.iloc[-4] = future2.iloc[-4].apply(lambda dt: dt.replace(day=1, month = 2 , year = 2020)) 
                future2.iloc[-3] = future2.iloc[-3].apply(lambda dt: dt.replace(day=7, month = 2, year = 2020)) 
                future2.iloc[-2] = future2.iloc[-2].apply(lambda dt: dt.replace(day=8, month = 2, year = 2020)) 
                future2.iloc[-1] = future2.iloc[-1].apply(lambda dt: dt.replace(day=14, month = 2, year = 2020)) 
                future2.sort_values(by = "ds", inplace=True, ascending=True)
                future2.drop_duplicates(subset=['ds'], inplace=True) 
                future2.reset_index(inplace = True, drop = True) 
                
                #Nos quedamos con los valores que nos interesa predecir
                
                future2 = future2[-4:]
                future2.reset_index(inplace = True, drop = True) 
                
                # Seguimos el modelo
                # future2['weekday'] = future2['ds'].apply(is_weekday)
                # future2['weekend'] = ~future2['ds'].apply(is_weekday)
                forecast2 = m2.predict(future2)
                #m2.plot(forecast2)  
                #plt.show() 
                
                #print("SEGUNDO FORECAST: ", forecast2[['ds', 'yhat']].tail())
                
                forecast3 = forecast2
                forecast3 = pd.DataFrame(forecast3[['yhat']])
                
                # cambiamos valores negativos por cero
                forecast3["yhat"].mask(forecast3["yhat"] < 0, 0, inplace = True) 
                forecast3.reset_index(drop=True, inplace=True)

                
                #metrica de evaluacion para el segundo modelo
                number2 = int(data_week.shape[0]) - 2 
                
                if int(data_week.shape[0]) < 100:
                    samples = []
                    # print("ESTE ES EL DATAWEEK: ", data_week)
                    # for index, row in data_week.iterrows():
                    #     print("Itero rows")
                    #     sampletime = row['ds']
                    #     delta = row['y']
                    #     print(row['ds'], row['y'])
                    #     samples.append(sampletime.date() + timedelta(days = 1))
                    #     samples.append(sampletime.date() - timedelta(days = 1))
                    #     df_add = pd.DataFrame({'ds': samples}, columns=['ds'])
                    #     df_add['y'] = 0
                    #     print("Dataset adicional: ", df_add)
                    #     data_week.append(df_add)
                    #     print(data_week)

                    number2 = 95
                else:
                    number2 = int(data_week.shape[0]) - 8 

            
                cv2 = cross_validation(m2, initial = f'{number2} days', period=f'{number2} days', horizon = '7 days')      
                df_pm2 = performance_metrics(cv2)
                metric2 = (df_pm2[:2]["rmse"].sum())/2
                metric2 = metric2*0.5
                
            
                w = metric2
                
                # Metrica final - Score
                
                def metrica(daily, week):
                    Metrica = 0.5*(daily) + 0.5*(week)
                    return Metrica
                
                #Juntamos los modelos
                

                frames1 = [result, forecast3[["yhat"]]]
                result1 = pd.concat(frames1)
                result1.reset_index(drop=True, inplace=True) 
                
                
                result1 = result1.drop(labels = "ds", axis = 1)
                result1.reset_index(drop=True, inplace=True) 
                result1['ID'] = ID
                result1.set_index('ID', inplace=True) 
                #result = result.drop('ID', 1)
                
                # Metricas            
                #creamos un array para guardar las metricas de dia, semanal, y el score final (d,w,metri) respectivamente
                metri = metrica(d,w) 
                metricas2 = pd.DataFrame(np.array([metri]))
                metricas2 = pd.concat([metricas2]) 
                metricas2 = metricas2.rename(columns={0:"metrica"})
                metricas2.reset_index(drop = True, inplace = True)
                
                
                #print("Result", result) 
                return result1, metricas2, proph 
            except Exception as e:
                print("No se ha incluido el ID: ", ID)
                print(e)
                id_errors.append(ID)
                return 0,0, False
            
        else: 
            return 0,0, proph  
        
    data2 = data1.copy()
    
    result, metricas2, proph = prophe(data2, data_week)
    
    if proph:
        
        dict_result[ID] = result['yhat'].to_numpy()
        dict_metricas[ID] = metricas2['metrica'].to_numpy()
        #print(dict_result)

        # final = final.append(result,  ignore_index=False)
        # final2 = final2.append(result2, ignore_index=False)
        ids_list.append(ID)
        if len(ids_list) % 100 == 0:
            print("Por ahora van perdidos:", id_errors)
            resul(dict_result, ids_list).to_csv('C:/hola/pred' +str(len(ids_list))+'.txt', sep = '|', header = False)
            resul(dict_result, ids_list).to_csv('C:/hola/metric' +str(len(ids_list))+'.txt', sep = '|', header = False)



        
# resul(dict_result, ids_list).to_csv('C:/Users/pulzara/Documents/Datathon/predicciones/pred_final.txt', sep = '|', header = False)
# resul(dict_metricas, ids_list).to_csv('C:/Users/pulzara/Documents/Datathon/predicciones/metricas.txt', sep = '|', header = False)
print("ESTO ES EL TOTAL DE IDS CON ERRORS: ", id_errors)
resul(dict_result, ids_list).to_csv('C:/hola/pred_final.txt', sep = '|', header = False)
resul(dict_metricas, ids_list).to_csv('C:/hola/metricas.txt', sep = '|', header = False) 